In [1]:
# 基本ライブラリ
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import numpy.random as rd
import gc
import multiprocessing as mp
import os
import sys
import pickle
from collections import defaultdict
from glob import glob
import math
from datetime import datetime as dt
from pathlib import Path
import scipy.stats as st
import re
import shutil
from tqdm import tqdm_notebook as tqdm
import datetime
ts_conv = np.vectorize(datetime.datetime.fromtimestamp) # 秒ut(10桁) ⇒ 日付

# グラフ描画系
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc

from matplotlib import animation as ani
from IPython.display import Image

plt.rcParams["patch.force_edgecolor"] = True
#rc('text', usetex=True)
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

#カラム内の文字数。デフォルトは50
pd.set_option("display.max_colwidth", 100)

#行数
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
#
pd.options.display.float_format = '{:,.5f}'.format

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
DATA_VERSION = "v003"
TRIAL_NO = "015"

In [3]:
importance_list = []
for i in range(8):
    path = f"../log/{DATA_VERSION}_{TRIAL_NO}/importance_{i}.csv"
    try:
        importance_df = pd.read_csv(path, index_col=0)
        importance_df = pd.concat([g.set_index("feature")[["importance"]].rename({"importance":f"fold_{i}"}, axis=1) for i, g in importance_df.groupby("fold")],axis=1)
        importance_df["ave"] = importance_df.mean(axis=1)
        importance_df["ratio"] = importance_df["ave"] / importance_df["ave"].sum()
        importance_df.sort_values("ratio", ascending=False ,inplace=True)
        importance_list += [importance_df]
        #break
    except Exception as e:
        print(e)

In [4]:
#np.arange(8)

In [5]:
importance_all_df = pd.concat([importance_list[i]["ratio"] for i in range(8)], axis=1)

importance_all_df.columns = [f"type_{i}" for i in range(8)]
importance_all_df["ave"] = importance_all_df.mean(axis=1)
importance_all_df.sort_values("ave", ascending=False ,inplace=True)
importance_all_df.to_excel(f"../log/{DATA_VERSION}_{TRIAL_NO}/importance_all.xlsx")
importance_all_df

/Users/kenichi.matsui/anaconda3/envs/kaggle_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,type_0,type_1,type_2,type_3,type_4,type_5,type_6,type_7,ave
oof_fc,0.02502,0.02463,0.02876,0.02158,0.02631,0.03311,0.02282,0.03308,0.02691
f006:dist_origin_mean,0.02267,0.03190,0.01692,0.02478,0.02499,0.01900,0.02179,0.02678,0.02360
abs_dist,0.02405,0.02771,0.01918,0.02226,0.02241,0.01677,0.01807,0.01757,0.02100
f006:dist_from_origin_0,0.02073,0.02099,0.01928,0.02101,0.02122,0.01887,0.01882,0.01943,0.02004
molecule_dist_mean,0.01892,0.02602,0.01393,0.02148,0.01898,0.01556,0.01943,0.01963,0.01924
molecule_dist_max,0.01870,0.02685,0.01453,0.02138,0.02047,0.01118,0.01891,0.01941,0.01893
oof_sd,0.01748,0.02081,0.01748,0.01680,0.02160,0.01750,0.01349,0.02406,0.01865
molecule_atom_1_dist_mean,0.01966,0.02215,0.01477,0.02055,0.01675,0.01521,0.02030,0.01611,0.01819
f006:dist_from_origin_1,0.01682,0.01693,0.01552,0.02207,0.01631,0.01581,0.02203,0.01594,0.01768
molecule_atom_index_0_x_1_std,0.01899,0.01908,0.01629,0.01727,0.01797,0.01587,0.01636,0.01689,0.01734


In [41]:
col=['a1_degree', 'a1_hybridization',
'a1_inring', 'a1_inring3', 'a1_inring4', 'a1_inring5', 'a1_inring6',
'a1_inring7', 'a1_inring8', 'a1_nb_h', 'a1_nb_o', 'a1_nb_c', 'a1_nb_n',
'a1_nb_na', 'a0_nb_degree', 'a0_nb_hybridization', 'a0_nb_inring',
'a0_nb_inring3', 'a0_nb_inring4', 'a0_nb_inring5', 'a0_nb_inring6',
'a0_nb_inring7', 'a0_nb_inring8', 'a0_nb_nb_h', 'a0_nb_nb_o',
'a0_nb_nb_c', 'a0_nb_nb_n', 'a0_nb_nb_na', 'x_a0_nb', 'y_a0_nb',
'z_a0_nb', 'a1_nb_degree', 'a1_nb_hybridization', 'a1_nb_inring',
'a1_nb_inring3', 'a1_nb_inring4', 'a1_nb_inring5', 'a1_nb_inring6',
'a1_nb_inring7', 'a1_nb_inring8', 'a1_nb_nb_h', 'a1_nb_nb_o',
'a1_nb_nb_c', 'a1_nb_nb_n', 'a1_nb_nb_na', 'x_a1_nb', 'y_a1_nb',
'z_a1_nb', 'dist_to_type_mean']

In [43]:
for c in col:
    print(c)

a1_degree
a1_hybridization
a1_inring
a1_inring3
a1_inring4
a1_inring5
a1_inring6
a1_inring7
a1_inring8
a1_nb_h
a1_nb_o
a1_nb_c
a1_nb_n
a1_nb_na
a0_nb_degree
a0_nb_hybridization
a0_nb_inring
a0_nb_inring3
a0_nb_inring4
a0_nb_inring5
a0_nb_inring6
a0_nb_inring7
a0_nb_inring8
a0_nb_nb_h
a0_nb_nb_o
a0_nb_nb_c
a0_nb_nb_n
a0_nb_nb_na
x_a0_nb
y_a0_nb
z_a0_nb
a1_nb_degree
a1_nb_hybridization
a1_nb_inring
a1_nb_inring3
a1_nb_inring4
a1_nb_inring5
a1_nb_inring6
a1_nb_inring7
a1_nb_inring8
a1_nb_nb_h
a1_nb_nb_o
a1_nb_nb_c
a1_nb_nb_n
a1_nb_nb_na
x_a1_nb
y_a1_nb
z_a1_nb
dist_to_type_mean


In [44]:
del_cols = """a0_nb_nb_c
a0_nb_inring5
a1_inring4
a1_inring5
a0_nb_nb_n
a1_nb_nb_c
a0_nb_inring4
a1_nb_nb_h
a1_nb_n
a0_nb_inring6
a1_inring6
a0_nb_nb_h
a1_nb_c
a1_inring3
a0_nb_nb_o
a1_nb_nb_n
a1_nb_inring5
a1_nb_nb_o
a1_nb_o
a1_nb_inring4
a1_inring7
a1_nb_inring3
a0_nb_inring
a1_nb_degree
a1_nb_h
a0_nb_inring7
a1_nb_inring6
a1_inring
F
a0_nb_inring3
a1_degree
a1_hybridization
a1_inring8
a0_nb_hybridization
a1_nb_hybridization
a1_nb_inring
a1_nb_inring7
a0_nb_inring8
a0_nb_degree
a1_nb_inring8
sp
a1_nb_na
a1_nb_nb_na
type
a0_nb_nb_na""".split("\n")

In [45]:
del_cols

['a0_nb_nb_c',
 'a0_nb_inring5',
 'a1_inring4',
 'a1_inring5',
 'a0_nb_nb_n',
 'a1_nb_nb_c',
 'a0_nb_inring4',
 'a1_nb_nb_h',
 'a1_nb_n',
 'a0_nb_inring6',
 'a1_inring6',
 'a0_nb_nb_h',
 'a1_nb_c',
 'a1_inring3',
 'a0_nb_nb_o',
 'a1_nb_nb_n',
 'a1_nb_inring5',
 'a1_nb_nb_o',
 'a1_nb_o',
 'a1_nb_inring4',
 'a1_inring7',
 'a1_nb_inring3',
 'a0_nb_inring',
 'a1_nb_degree',
 'a1_nb_h',
 'a0_nb_inring7',
 'a1_nb_inring6',
 'a1_inring',
 'F',
 'a0_nb_inring3',
 'a1_degree',
 'a1_hybridization',
 'a1_inring8',
 'a0_nb_hybridization',
 'a1_nb_hybridization',
 'a1_nb_inring',
 'a1_nb_inring7',
 'a0_nb_inring8',
 'a0_nb_degree',
 'a1_nb_inring8',
 'sp',
 'a1_nb_na',
 'a1_nb_nb_na',
 'type',
 'a0_nb_nb_na']

In [48]:
use_cols = pd.read_csv("../log/v003_009/use_cols.csv", index_col=0)

In [52]:
for c in [c for c in use_cols["columns"] if c not in del_cols]:
    print(c)

molecule_atom_index_0_dist_min
molecule_atom_index_0_dist_max
molecule_atom_index_1_dist_min
molecule_atom_index_0_dist_mean
molecule_atom_index_0_dist_std
dist
abs_dist
x_0
y_0
z_0
x_1
y_1
z_1
molecule_atom_index_1_dist_std
molecule_atom_index_1_dist_max
molecule_atom_index_1_dist_mean
molecule_atom_index_0_dist_max_diff
molecule_atom_index_0_dist_max_div
molecule_atom_index_0_dist_std_diff
molecule_atom_index_0_dist_std_div
atom_0_couples_count
molecule_atom_index_0_dist_min_div
molecule_atom_index_1_dist_std_diff
molecule_atom_index_0_dist_mean_div
atom_1_couples_count
molecule_atom_index_0_dist_mean_diff
molecule_couples
atom_index_1
molecule_dist_mean
molecule_atom_index_1_dist_max_diff
molecule_atom_index_0_y_1_std
molecule_atom_index_1_dist_mean_diff
molecule_atom_index_1_dist_std_div
molecule_atom_index_1_dist_mean_div
molecule_atom_index_1_dist_min_diff
molecule_atom_index_1_dist_min_div
molecule_atom_index_1_dist_max_div
molecule_atom_index_0_z_1_std
molecule_type_dist_std_di

In [53]:
rdkit_cols = ['id', 'a1_degree', 'a1_hybridization',
              'a1_inring', 'a1_inring3', 'a1_inring4', 'a1_inring5', 'a1_inring6',
              'a1_inring7', 'a1_inring8', 'a1_nb_h', 'a1_nb_o', 'a1_nb_c', 'a1_nb_n',
              'a1_nb_na', 'a0_nb_degree', 'a0_nb_hybridization', 'a0_nb_inring',
              'a0_nb_inring3', 'a0_nb_inring4', 'a0_nb_inring5', 'a0_nb_inring6',
              'a0_nb_inring7', 'a0_nb_inring8', 'a0_nb_nb_h', 'a0_nb_nb_o',
              'a0_nb_nb_c', 'a0_nb_nb_n', 'a0_nb_nb_na', 'x_a0_nb', 'y_a0_nb',
              'z_a0_nb', 'a1_nb_degree', 'a1_nb_hybridization', 'a1_nb_inring',
              'a1_nb_inring3', 'a1_nb_inring4', 'a1_nb_inring5', 'a1_nb_inring6',
              'a1_nb_inring7', 'a1_nb_inring8', 'a1_nb_nb_h', 'a1_nb_nb_o',
              'a1_nb_nb_c', 'a1_nb_nb_n', 'a1_nb_nb_na', 'x_a1_nb', 'y_a1_nb',
              'z_a1_nb', 'dist_to_type_mean']

In [54]:
[c for c in rdkit_cols if c not in del_cols]

['id',
 'x_a0_nb',
 'y_a0_nb',
 'z_a0_nb',
 'x_a1_nb',
 'y_a1_nb',
 'z_a1_nb',
 'dist_to_type_mean']